In [1]:
import numpy as np
from scipy.stats import chi2_contingency
import statsmodels.stats.multitest as multitest
import matplotlib.pyplot as plt

# This code below would perform false discovery rate correction to the label-label dependency p_value matrix
def fdr_correct(p_value):
  p_value_array = p_value[np.tril_indices(len(p_value))]
  corrected_p_value = multitest.multipletests(p_value_array,0.01,method='fdr_bh')[1]
  inds = np.tril_indices(len(p_value))
  p_value[inds] = corrected_p_value
  inds = np.triu_indices(len(p_value))
  p_value[inds] = corrected_p_value[::-1]
###

def get_label_depend_p_value(y,alpha, is_fdr_correct=True):
  count_11 = np.matmul(y.transpose(), y) #Aij is sum of 11 of label i and label j
  count_10 = np.matmul(y.transpose(), 1 - y) 
  count_01 = np.matmul(1 - y.transpose(), y)
  count_00 = np.matmul(1 - y.transpose(), 1 - y)

  # chi2 testing on pairwise dependencies
  num_vars = y.shape[1] # num labels
  p_value = np.ones([num_vars,num_vars])
  for i in range(num_vars):
      for j in range(num_vars):
          
          c = np.array([[count_11[i, j], count_10[i, j]], [count_01[i, j], count_00[i, j]]])
          
          if (np.sum(c, 0) > 0).all() and (np.sum(c, 1) > 0).all():
              _, p_value[i, j], _, _ = chi2_contingency(c)
          else:
              p_value[i,j] = 1
          if(i==50 and j==50):
            print(c)

  uncorrected_p_value = p_value.copy()
  if(is_fdr_correct):
    fdr_correct(p_value)
  skeleton = 1*(p_value <= alpha)

  return uncorrected_p_value, p_value,skeleton

def _calc_prob(y):
  marginal_1 = np.sum(y, 0)/len(y) # vector containing probability a given label is 1
  marginal_0 = 1-marginal_1
  joint_11 = np.matmul(y.transpose(), y)/len(y) # matrix prob that 11 occurs in ith, jth position in a given row
  joint_01 = np.matmul(1-y.transpose(), y)/len(y)
  joint_10 = np.matmul(y.transpose(), 1-y)/len(y)
  joint_00 = np.matmul(1-y.transpose(), 1-y)/len(y)
  conditional_11 = joint_11/marginal_1  # probability of dim=0 (row) given dim=1 (column)
  conditional_01 = joint_01/marginal_1 
  conditional_10 = joint_10/marginal_0
  conditional_00 = joint_00/marginal_0
  unscaled_11=np.matmul(y.transpose(), y)
  unscaled_marginal_1 = np.sum(y, 0)
  return {'marginal_1': marginal_1, 'marginal_0': marginal_0,
          'joint_11': joint_11, 'conditional_11': conditional_11,
          'joint_01': joint_01, 'conditional_01': conditional_01,
          'joint_10': joint_10, 'conditional_10': conditional_10,
          'joint_00': joint_00, 'conditional_00': conditional_00,
          'unscaled_joint_11':unscaled_11,'unscaled_marginal_1':unscaled_marginal_1
          }
  
class Dataset:
  def __init__(self, y, alpha, is_fdr_correct=True):
    self.y = y
    self.uncorrected_p_value, self.p_value, self.skeleton = get_label_depend_p_value(y,alpha,is_fdr_correct)
    p = _calc_prob(y)
    self.marginal = p['marginal_1']
    self.conditional = p['conditional_11'] # the chi2 test tells us whether the difference between conditional and marginal is significant
    self.pulling = self.skeleton * (self.conditional > self.marginal[:, None]) # matrix, where each element is 1 if the presence if ith label increases chance of jth label AND chi2 test satisfied
    self.pushing = self.skeleton * (self.conditional < self.marginal[:, None])
    self.pushing_to_pulling_ratio = np.count_nonzero(self.pushing)/np.count_nonzero(self.pulling)
    self.alpha = alpha
    self.num_vars = self.y.shape[1]



  def plot_dependency_strength(self,relation,fig_dir=None):
    plt.figure(figsize=(8,8),dpi=320)
    if (relation=='pulling'):
      plt.matshow(self.pulling * (np.divide(self.conditional - self.marginal[:, None],self.marginal[:, None])),fignum=0,vmax=10), plt.colorbar()
    if (relation=='pushing'):
      plt.matshow(self.pushing * (np.divide(self.conditional - self.marginal[:, None],self.marginal[:, None])),fignum=0,vmax=1), plt.colorbar()
    plt.xlabel('strength of ' + ' ' +relation)
    
    if (fig_dir):
      plt.savefig(fig_dir)
    plt.clf()
  def plot_instance_label_matrix(self,fig_dir=None):
    plt.figure(figsize=(12,8))
    plt.spy(self.y,aspect='auto')
    if (fig_dir):
      plt.savefig(fig_dir)
    plt.clf()
  def plot_ratio_of_dependent_pairs(self, relation,fig_dir=None):
    overall_ratio = []
    pulling_ratio = []
    pushing_ratio = []
    plot_x = np.linspace(0.002,0.9,50)
    num_vars = self.y.shape[1]
    for num in plot_x:

      overall_ratio.append((np.count_nonzero(self.p_value<num)-num_vars)/(self.p_value.size-num_vars))
      pulling_ratio.append((np.count_nonzero(np.logical_and(self.p_value<num,self.conditional > self.marginal[:, None]))-num_vars)/(self.p_value.size-num_vars))
      pushing_ratio.append((np.count_nonzero(np.logical_and(self.p_value<num,self.conditional < self.marginal[:, None]))-num_vars)/(self.p_value.size-num_vars))

    if (relation=='pulling'):
      plt.plot(plot_x,pulling_ratio)
      plt.xlabel('alpha')
      plt.ylabel('ratio of dependent labels [pulling]')
    if (relation=='pushing'):
      plt.plot(plot_x,pushing_ratio)
      plt.xlabel('alpha')
      plt.ylabel('ratio of dependent labels [pushing]')
    if (relation=='overall'):
      plt.plot(plot_x,overall_ratio)
      plt.xlabel('alpha')
      plt.ylabel('ratio of dependent labels [overall]')
    
    if (fig_dir):
      plt.savefig(fig_dir)
    plt.clf()
  def print_stats(self): 
    num_depend = np.count_nonzero(np.logical_and(self.p_value<self.alpha,np.tril(self.p_value,-1)))
    uncorrected_num_depend = np.count_nonzero(np.logical_and(self.uncorrected_p_value<self.alpha,np.tril(self.p_value,-1)))
    cardinality = np.mean(np.sum(self.y,1))
    print('Label cardinality: ' + str(cardinality))
    print('Label density: ' + str(cardinality/self.num_vars))
    print('Number of dependent pairs less than alpha: ' + str(num_depend))
    print('Proportion of dependent pairs less than alpha: ' + str(num_depend/(self.p_value.size-self.num_vars)*2))
    print('Proportion of dependent pairs less than alpha using uncorrected p-values: ' + str(uncorrected_num_depend/(self.p_value.size-self.num_vars)*2))
    print('Pushing to Pulling ratio: '+ str(self.pushing_to_pulling_ratio))



In [2]:
import arff

from scipy import sparse
decoder = arff.ArffDecoder()
d = decoder.decode(open('enron-train.arff', 'r'), encode_nominal=True, return_type=arff.COO)
data = d['data'][0]
row = d['data'][1]
col = d['data'][2]
matrix = sparse.coo_matrix((data, (row, col)), shape=(max(row)+1, max(col)+1))
y = matrix.A[:,1001:]

In [3]:
data1 = Dataset(y,0.05,is_fdr_correct=True)
data1.plot_dependency_strength('pulling','enron_pulling.pdf')
data1.plot_dependency_strength('pushing','enron_pushing.pdf')
data1.plot_instance_label_matrix('enron_instance_label.pdf')
data1.plot_ratio_of_dependent_pairs('pulling','enron_pulling_alpha.pdf')
data1.plot_ratio_of_dependent_pairs('pushing','enron_pushing_alpha.pdf')
data1.plot_ratio_of_dependent_pairs('overall','enron_overall_alpha.pdf')
data1.print_stats()

[[  15    0]
 [   0 1108]]


c:\users\david\anaconda3\envs\graphenv\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in true_divide
c:\users\david\anaconda3\envs\graphenv\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in true_divide
c:\users\david\anaconda3\envs\graphenv\lib\site-packages\ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in true_divide
c:\users\david\anaconda3\envs\graphenv\lib\site-packages\ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in true_divide


Label cardinality: 3.3864648263579697
Label density: 0.06389556276147113
Number of dependent pairs less than alpha: 220
Proportion of dependent pairs less than alpha: 0.15965166908563136
Proportion of dependent pairs less than alpha using uncorrected p-values: 0.23947750362844702
Pushing to Pulling ratio: 1.6703296703296704


<Figure size 2560x2560 with 0 Axes>

<Figure size 2560x2560 with 0 Axes>

<Figure size 864x576 with 0 Axes>